# **Preprocessing and Modeling**

#### *Imports and Read in Data*

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer

In [3]:
reddit = pd.read_csv('../data/reddit_cleaned.csv')
reddit.head(2)

,subreddit,author,selftext,created_utc,author_premium,is_video,score,title,upvote_ratio,num_comments,post_char_length,post_word_count
0,0,Jay_Subabove,"This batter is hitting .191 with 3HR, 12 RBI, ...",1655862592,False,False,1,Am I crazy?,1.0,0,263,53
1,0,Stress_Factor,"Not a Yankees fan, but modern day record looki...",1655856890,False,False,1,MLB Record (Wins),1.0,0,96,19


------
## Preprocessing

In [5]:
# using a function to remove html which seemed to be scattered throughout based on EDA
# this code was adapted from the breakfast hour NLP practice for week 5

def remove_html(post):
    '''function to remove html and lowercase all text'''
    post = post.lower()
    no_html = BeautifulSoup(post).text
    
    return no_html

In [6]:
reddit['clean_text'] = reddit['selftext'].apply(remove_html)
reddit.head(2)

/Users/petermurphy/opt/anaconda3/envs/dsi/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/petermurphy/opt/anaconda3/envs/dsi/lib/python3.9/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


,subreddit,author,selftext,created_utc,author_premium,is_video,score,title,upvote_ratio,num_comments,post_char_length,post_word_count,clean_text
0,0,Jay_Subabove,"This batter is hitting .191 with 3HR, 12 RBI, ...",1655862592,False,False,1,Am I crazy?,1.0,0,263,53,"this batter is hitting .191 with 3hr, 12 rbi, ..."
1,0,Stress_Factor,"Not a Yankees fan, but modern day record looki...",1655856890,False,False,1,MLB Record (Wins),1.0,0,96,19,"not a yankees fan, but modern day record looki..."


In [8]:
# creating functions that stem and lemmatize text - to be used as hyperparameters
# this code was adapted from the breakfast hour NLP practice for week 5
# lemmatize first

def lemmatize(post):
    '''
    Function splits the text data,
    lemmatizes it, and rejoins
    '''
    post_split = post.split()
    lemmatizer = WordNetLemmatizer
    
    return " ".join([lemmatizer.lemmatize(word) for word in post_split])

In [9]:
# now for stemming

def stem(post):
    '''Same framework applied as lemmatize'''
    post_split = post.split()
    p_stemmer = PorterStemmer
    
    return " ".join([p_stemmer.stem(word) for word in post_split])

------
## Modeling

In [13]:
# starting with simple models - just text column and default hyperparameters
# will tune once there appears to be a pipeline that works best

X = reddit['clean_text']
y = reddit['subreddit']

### **Defining the baseline accuracy**

In [12]:
y.value_counts(normalize=True)

0    0.510562
1    0.489438
Name: subreddit, dtype: float64

*hoping to beat the 51% baseline accuracy*

In [14]:
# splitting the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
print(X_train.shape)
print(X_test.shape)

(1668,)
(557,)


#### 1) Count Vectorizer and Naive Bayes (defaults)

#### 2) Tfidf Vectorizer and Naive Bayes (defaults)

#### 3) Count Vectorizer and Logistic Regression (defaults)

#### 4) Tfidf  Vectorizer and Logistic Regression (defaults)

play with the hyperparameters

Gridsearching for best hyperparameters for either each model or just the best default one, depending on how they do

In [15]:
#build a pipeline to run these models
#Don't pass gridsearch all paramaters to test at once (would overload)
ngram_range
max_iterations
max_features
max_df
min_df
preprocessor (offer it my functions for stem and lemmatize)
tokenizer


SyntaxError: invalid syntax (1270235564.py, line 8)

In [ ]:
#consider different stopwords to use 
# Print English stopwords.
print(stopwords.words('english'))

In [ ]:
type(stopwords.words('english'))

In [ ]:
# Remove stopwords from "spam_tokens."
no_stop_words = [token for token in spam_tokens if token not in stopwords.words('english')]

In [ ]:
# Check it
print(no_stop_words)

In [ ]:
Checking model
can plot confusion matrix
look at accuracy 
and f1 score(don't care about weighting sensitivity or specificity too much)
maybe precision as well (tp/(tp+fp))